In [3]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/MyDrive/Colab Notebooks/CoCrNi_experiment

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/MyDrive/Colab Notebooks/CoCrNi_experiment


In [4]:
pip install fast_histogram

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 kB 4.9 MB/s eta 0:00:00


In [5]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import psutil
n_phys_cores=psutil.cpu_count(logical=False)
from scipy import spatial
import datetime
from os import listdir
from joblib import Parallel, delayed


# custom modules
import zsdm_utils_v2 as zsdm_utils
# import datasphere.populate as dsp

CSRO prediction

In [6]:
#%% generate exp ZSDM
folder = os.path.join(os.getcwd(), 'data')
csv_paths = zsdm_utils.get_csv_paths(folder)
data = pd.read_csv(csv_paths[0],
                    names=['x', 'y', 'z', 'Da'])
#print(data)
rrange_file = 'R5096_71247_100.RRNG'
ions, rrngs = zsdm_utils.read_rrng(rrange_file)
#print(rrngs)

element_1_name = 'Co'
element_2_name = 'Cr'
element_3_name = 'Ni'
element_1_range = rrngs[rrngs['comp']==element_1_name+':1']
element_2_range = rrngs[rrngs['comp']==element_2_name+':1']
element_3_range = rrngs[rrngs['comp']==element_3_name+':1']
#print(element_1_range)
#print(element_2_range)

element_1 = zsdm_utils.atom_filter(data, element_1_range)
element_2 = zsdm_utils.atom_filter(data, element_2_range)
element_3 = zsdm_utils.atom_filter(data, element_3_range)

data_min = data.min()
data_max = data.max()

#print(data_min)
#print(data_max)
data_min['c'], data_min['b'], data_min['a'] = -120, -10, -10  #nm
data_max['c'], data_max['b'], data_max['a'] = 0, 10, 10   #nm

# z_limit, y_limit, x_limit,  = -90, -10, -10
# scanning parameters 
voxel = np.array([1],dtype=np.float64)
#print("voxel shape {}".format(voxel.shape))
stride = 0.5
#%% Model predictions
for element_name_chosen in (element_1_name, element_2_name, element_3_name):
    nzSDM_exp_preprocessing_parallel_2 = np.load('nzSDM_exp_preprocessing_2_'+element_name_chosen+element_name_chosen+'_%s_%s_10_10_1_0.5.npy'
                                                 %(int(data_min['c']), int(data_max['c'])))
    w1=nzSDM_exp_preprocessing_parallel_2.shape[0]
    len1=93
    x_exp = np.reshape(nzSDM_exp_preprocessing_parallel_2, (w1, len1))
    x_exp_dim = np.reshape(x_exp[:,:], x_exp[:,:].shape+(1,))
    
    y_exp_prediction_0, y_exp_prediction_1 = np.zeros((w1, 1)), np.zeros((w1, 1)) #revised into four classes
    
    for k in range(5):
        #calling model 
        print('k=', k)
        model = tf.keras.models.load_model('saved_models_1CNN_layer_0923/model_%s.h5'%(k)) 
        model.summary()
         
        # # Test data
        y_exp_prediction_single = model.predict(x_exp_dim[:,:,:])
        y_exp_prediction_0 = np.concatenate((y_exp_prediction_0, y_exp_prediction_single[:, 0].reshape(-1,1)), axis=1)
        y_exp_prediction_1 = np.concatenate((y_exp_prediction_1, y_exp_prediction_single[:, 1].reshape(-1,1)), axis=1)
        # y_exp_prediction_2 = np.concatenate((y_exp_prediction_2, y_exp_prediction_single[:, 2].reshape(-1,1)), axis=1)
        # y_exp_prediction_3 = np.concatenate((y_exp_prediction_3, y_exp_prediction_single[:, 3].reshape(-1,1)), axis=1)
        
    y_exp_prediction_0, y_exp_prediction_1 = y_exp_prediction_0[:, 1:], y_exp_prediction_1[:, 1:]
    y_exp_prediction_0_ave, y_exp_prediction_0_std = np.mean(y_exp_prediction_0, axis=1), np.std(y_exp_prediction_0, axis=1)
    y_exp_prediction_1_ave, y_exp_prediction_1_std = np.mean(y_exp_prediction_1, axis=1), np.std(y_exp_prediction_1, axis=1)
    # y_exp_prediction_2_ave, y_exp_prediction_2_std = np.mean(y_exp_prediction_2, axis=1), np.std(y_exp_prediction_2, axis=1)
    # y_exp_prediction_3_ave, y_exp_prediction_3_std = np.mean(y_exp_prediction_3, axis=1), np.std(y_exp_prediction_3, axis=1)
    
    y_exp_predictions = np.concatenate((y_exp_prediction_0_ave.reshape(-1,1), y_exp_prediction_1_ave.reshape(-1,1)), axis=1)
    
    np.save('CoCrNi_test_data_'+element_name_chosen+element_name_chosen+'_%s_%s_%s_%s_%s_%s.npy'%(int(data_min['c']), int(data_max['c']), 
                                                                            int(data_max['b']), int(data_max['a']), 1, stride), y_exp_predictions)
    


/content/gdrive/MyDrive/Colab Notebooks/CoCrNi_experiment/zsdm_utils_v2.py:82: FutureWarning: Boolean inputs to the `inclusive` argument are deprecated in favour of `both` or `neither`.
  x['Da'].between(Atom_range['lower'][i], Atom_range['upper'][i], inclusive=True)
/content/gdrive/MyDrive/Colab Notebooks/CoCrNi_experiment/zsdm_utils_v2.py:82: FutureWarning: Boolean inputs to the `inclusive` argument are deprecated in favour of `both` or `neither`.
  x['Da'].between(Atom_range['lower'][i], Atom_range['upper'][i], inclusive=True)


ATOM TOTAL = 451319
ATOM TOTAL = 497758


/content/gdrive/MyDrive/Colab Notebooks/CoCrNi_experiment/zsdm_utils_v2.py:82: FutureWarning: Boolean inputs to the `inclusive` argument are deprecated in favour of `both` or `neither`.
  x['Da'].between(Atom_range['lower'][i], Atom_range['upper'][i], inclusive=True)


ATOM TOTAL = 428632
k= 0
Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_22 (Conv1D)          (None, 84, 16)            176       
                                                                 
 dropout_11 (Dropout)        (None, 84, 16)            0         
                                                                 
 max_pooling1d_11 (MaxPoolin  (None, 42, 16)           0         
 g1D)                                                            
                                                                 
 flatten_11 (Flatten)        (None, 672)               0         
                                                                 
 dense_22 (Dense)            (None, 256)               172288    
                                                                 
 dense_23 (Dense)            (None, 2)                 514       
                            

CSRO extraction

In [12]:
def SRO_extract(i_threshold):
    lower, upper = threshold+i_threshold*ite, threshold+(i_threshold+1)*ite
    print('Threshold is from ', lower, ' to ', upper)
    dspp_filter = dspp.loc[(dspp['d'] > lower) & (dspp['d'] <=upper) , ['a','b','c','d']]  
    tree = []
    tree = spatial.cKDTree(df_1[['a','b','c']])
    index_voxel_sphere = tree.query_ball_point(dspp_filter[['a','b','c']], stride/2*1.414)     
    myList = range(0,len(index_voxel_sphere))
    pre_extract_v1 = pd.DataFrame(np.arange(4).reshape(1,4),columns=['a','b','c','d'])
    for i in myList:
        df_2 = df_1.iloc[index_voxel_sphere[i,]]
        pre_extract_v1 = pre_extract_v1.append(df_2)
    pre_extract_v1 = pre_extract_v1[1:].drop_duplicates().values
    np.savetxt('pre_extract\\'+ element_name_chosen+element_name_chosen +
               '_10_10_%s_%snm_above_%s_%s_win_%s_stride_%s.csv'
               %(int(data_min['c']), int(data_max['c']), lower, upper, voxel, stride) 
               , pre_extract_v1, delimiter=',', fmt='%.3f')
    return pre_extract_v1


def SRO_extract_only_lower(lower):
    print('Threshold is above ', lower)
    dspp_filter = dspp.loc[(dspp['d'] > lower) , ['a','b','c','d']]  
    tree = []
    tree = spatial.cKDTree(df_1[['a','b','c']])
    index_voxel_sphere = tree.query_ball_point(dspp_filter[['a','b','c']], stride/2*1.414)     
    myList = range(0,len(index_voxel_sphere))
    pre_extract_v1 = pd.DataFrame(np.arange(4).reshape(1,4),columns=['a','b','c','d'])
    for i in myList:
        df_2 = df_1.iloc[index_voxel_sphere[i,]]
        pre_extract_v1 = pre_extract_v1.append(df_2)
    pre_extract_v1 = pre_extract_v1[1:].drop_duplicates().values
    np.savetxt('pre_extract\\'+ element_name_chosen+element_name_chosen +
               '_10_10_%s_%snm_above_%s_win_%s_stride_%s.csv'
               %(int(data_min['c']), int(data_max['c']), lower, voxel, stride) 
               , pre_extract_v1, delimiter=',', fmt='%.3f')
    return pre_extract_v1


In [13]:
#%% Input APT data
folder = 'data'

# for filename in tqdm(os.listdir(folder)):    
#     print(filename)
#     pos = readpos(folder+'/'+filename)
#     dpos = pd.DataFrame({'x':pos['x'],
#                                 'y': pos['y'],
#                                 'z': pos['z'],
#                                 'Da': pos['m']})
#     dpos.to_csv(folder+'/'+'{}.csv'.format(filename), index=False)  

data_name = [file for file in listdir(folder) if file.endswith('.csv')]
data_name = data_name[0]
df_1 = pd.read_csv(folder+'/'+data_name, names=['a','b','c','d'])
# df_1.columns =
#%% get max, min of each column
data_min = df_1.min()
data_min['c'], data_min['b'], data_min['a'] = -120, -10, -10  #nm
data_max = df_1.max()
data_max['c'], data_max['b'], data_max['a'] = 0, 10, 10   #nm

#%% define voxel and stride 
voxel = 1   #nm   delta
stride = 0.5   #nm   delta_voxels
count = 0*8
threshold = 2.75
ite = 0.25

element_1_name = 'Co'
element_2_name = 'Cr'
element_3_name = 'Ni'
# dict = {'CoCo': 1} #dict = {'B2': 1, 'D03': 2}


for element_name_chosen in (element_1_name, element_2_name, element_3_name):
    #Input the probability of each voxel
    data_predictions = np.load('CoCrNi_test_data_'+element_name_chosen+element_name_chosen+'_%s_%s_%s_%s_%s_%s.npy'%(
                                int(data_min['c']), int(data_max['c']), int(data_max['b']), int(data_max['a']), voxel, stride))
    # row_number = data.shape[0]
    # detect_eff = 0.52
    n_dim = data_predictions.shape[1] #0 matrix 1 CSRO
    #%% reshape data_predictions
    img_dim_a = int((data_max['a']-data_min['a'])/1)
    img_dim_b = int((data_max['b']-data_min['b'])/1)
    img_dim_c = int((data_max['c']-data_min['c'])/1)   
    conv_shape_a = int((img_dim_a-voxel)/stride)+1
    conv_shape_b = int((img_dim_b-voxel)/stride)+1
    conv_shape_c = int((img_dim_c-voxel)/stride)+1
    
    conv_probability = np.zeros([conv_shape_c, conv_shape_b, conv_shape_a])   
    
    # for sro_type in dict:   
    #     print(dict[sro_type])
    conv_probability[:,:,:] = np.reshape(data_predictions[:, 1], (conv_shape_c, conv_shape_b, conv_shape_a))
    # print(conv_probability[:,:,:,len1])
    #%% calculate the probability of each voxel (stride*stride*4) 
    starttime = datetime.datetime.now()
    
    pixel_probability = np.zeros([int(img_dim_c/stride), int(img_dim_b/stride), int(img_dim_a/stride)])    #1
    pixel_z = 0
    pixel_y = 0
    pixel_x = 0
    count = 0
    intervial= int(voxel/stride)
    # print ('count', count)
    # print('pixel_x=', pixel_x)
    # print('pixel_y=', pixel_y)
    # print('pixel_z=', pixel_z)
    # for pixel_dim in range (n_dim):
    #     print ('Dimension=', pixel_dim)
    for data_Z in np.arange(int(data_min['c']), int(data_max['c']), stride):
        if data_Z+stride > data_max['c']:
            continue
        else:
            for data_Y in np.arange(int(data_min['b']),int(data_max['b']), stride):
                if data_Y+stride > data_max['b']:
                    continue
                else:
                    for data_X in np.arange(int(data_min['a']), int(data_max['a']), stride):
                        if data_X+stride > data_max['a']:
                            continue
                        else:
    
                            for k in range (pixel_z, pixel_z-intervial, -1):
                                if k < 0 or k >= conv_shape_c: 
                                    continue
                                else:
                                    for j in range (pixel_y, pixel_y-intervial, -1):
                                        if j < 0 or j >= conv_shape_b:
                                            continue
                                        else:                                       
                                            for i in range (pixel_x, pixel_x-intervial, -1):
                                                if i < 0 or i >= conv_shape_a:
                                                    continue
                                                else:
                                                
                                                    temp = conv_probability[k, j, i]
                                                    pixel_probability[pixel_z, pixel_y, pixel_x] = pixel_probability[pixel_z, pixel_y, pixel_x] + temp
                                                    # print ('i=',i, "j=", j, "k=", k)
                                                    count = count + 1
                            pixel_probability[pixel_z, pixel_y, pixel_x] = pixel_probability[pixel_z, pixel_y, pixel_x]
                            # print ('count', count)
                            count=0        
                            # print ("------>>")
                            if pixel_x == int(img_dim_a/stride-1):
                                pixel_x = 0  
                            else:
                                pixel_x = pixel_x + 1
                            # print('pixel_x=', pixel_x)
                            # print('pixel_y=', pixel_y)
                            # print('pixel_z=', pixel_z)
                    if pixel_y == int(img_dim_b/stride-1):
                        pixel_y = 0                        
                    else:
                        pixel_y = pixel_y + 1
                    # print('pixel_x=', pixel_x)
                    # print('pixel_y=', pixel_y)
                    # print('pixel_z=', pixel_z)
            if pixel_z == int(img_dim_c/stride-1):
                pixel_z = 0                        
            else:
                pixel_z = pixel_z + 1
            # print('pixel_x=', pixel_x)
            # print('pixel_y=', pixel_y)
            # print('pixel_z=', pixel_z)
    endtime = datetime.datetime.now()
    print ('1st running time is', endtime-starttime)
    #%% Corresponding the probability of each small voxel with each data point, and then saved into .txt file
    starttime = datetime.datetime.now()
    
    dim_a = int((data_max['a']-data_min['a'])/stride)
    dim_b = int((data_max['b']-data_min['b'])/stride)
    dim_c = int((data_max['c']-data_min['c'])/stride)
    pixel_probability_flatten=np.reshape(pixel_probability, (dim_a*dim_b*dim_c, 1))
    np.save('pixel_probability_flatten_'+element_name_chosen+element_name_chosen, pixel_probability_flatten)
    
    try:
        data_sphere_points = np.load('data_sphere_points_cocrni_%s_%s.npy'%(int(data_min['c']), int(data_max['c'])))
    except:
        data_Z_list = list(np.arange(int(data_min['c']), int(data_max['c']), stride))
        data_Y_list = list(np.arange(int(data_min['b']), int(data_max['b']), stride))
        data_X_list = list(np.arange(int(data_min['a']), int(data_max['a']), stride))
        
        data_sphere_points = np.zeros((1,3))
        for data_Z, data_Y, data_X in product(data_Z_list, data_Y_list, data_X_list):
            if data_Z+stride > data_max['c'] or data_Y+stride > data_max['b'] or data_X+stride > data_max['a']:
                continue
            else:
                temp = np.array([data_X+stride/2, data_Y+stride/2, data_Z+stride/2]).reshape((1,3)) 
                data_sphere_points = np.concatenate((data_sphere_points, temp), axis=0)
        data_sphere_points = data_sphere_points[1:]
        np.save ('data_sphere_points_cocrni_%s_%s.npy'%(int(data_min['c']), int(data_max['c'])), data_sphere_points)
        
    data_sphere_points_probability = np.concatenate((data_sphere_points, pixel_probability_flatten), axis=1)
    dspp = pd.DataFrame(data_sphere_points_probability, columns=['a','b','c','d'])
    
    #output using different thresholds
    # myList = range(0,6)
    # pre_extract = Parallel(n_jobs=-1, verbose=1)(delayed(SRO_extract)(i) for i in myList)
    
    SRO_extract_only_lower(3.75)  

    endtime = datetime.datetime.now()
    print ('2nd running time is', endtime-starttime)

1st running time is 0:00:05.686399
Threshold is above  3.75


Streaming output truncated to the last 5000 lines.
<ipython-input-12-5bdf864f6e78>:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pre_extract_v1 = pre_extract_v1.append(df_2)
<ipython-input-12-5bdf864f6e78>:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pre_extract_v1 = pre_extract_v1.append(df_2)
<ipython-input-12-5bdf864f6e78>:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pre_extract_v1 = pre_extract_v1.append(df_2)
<ipython-input-12-5bdf864f6e78>:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pre_extract_v1 = pre_extract_v1.append(df_2)
<ipython-input-12-5bdf864f6e78>:31: FutureWarning: The frame.append method is depreca

2nd running time is 0:00:06.478521
1st running time is 0:00:05.384402
Threshold is above  3.75


Streaming output truncated to the last 5000 lines.
<ipython-input-12-5bdf864f6e78>:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pre_extract_v1 = pre_extract_v1.append(df_2)
<ipython-input-12-5bdf864f6e78>:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pre_extract_v1 = pre_extract_v1.append(df_2)
<ipython-input-12-5bdf864f6e78>:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pre_extract_v1 = pre_extract_v1.append(df_2)
<ipython-input-12-5bdf864f6e78>:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pre_extract_v1 = pre_extract_v1.append(df_2)
<ipython-input-12-5bdf864f6e78>:31: FutureWarning: The frame.append method is depreca

2nd running time is 0:00:05.789793
1st running time is 0:00:06.017648
Threshold is above  3.75


Streaming output truncated to the last 5000 lines.
<ipython-input-12-5bdf864f6e78>:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pre_extract_v1 = pre_extract_v1.append(df_2)
<ipython-input-12-5bdf864f6e78>:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pre_extract_v1 = pre_extract_v1.append(df_2)
<ipython-input-12-5bdf864f6e78>:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pre_extract_v1 = pre_extract_v1.append(df_2)
<ipython-input-12-5bdf864f6e78>:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pre_extract_v1 = pre_extract_v1.append(df_2)
<ipython-input-12-5bdf864f6e78>:31: FutureWarning: The frame.append method is depreca

2nd running time is 0:00:06.673851
